In [1]:
import win32com.client
import os
import time

In [2]:
# -------
def fn_filepath_expected(str_input_filepath, 
                         str_input_extension, 
                         str_output_extension):

    # From a given input path of a HEC-RAS file, determine if the run has created
    # the required Linux run inputs
    
    # Split the file path into directory, base filename, and extension
    directory, filename = os.path.split(str_input_filepath)
    base_filename, extension = os.path.splitext(filename)

    # Example: Replace 'gXX' with 'bXX' in the extension
    mod_extension = extension.replace(str_input_extension, str_output_extension)

    # Construct the file path
    str_ammended_filepath = os.path.join(directory, base_filename + mod_extension)
    
    return(str_ammended_filepath)
# -------

In [3]:
# --- Input ---
str_ras_projectpath = r'E:\HECRAS_2D_12070205\base_model_20240414_copy\BLE_LBSG_501.prj'

# to set the version of the HEC-RAS controller
str_hecras_controller = 'RAS610.HECRASController'
# --- Input ---

hec = win32com.client.Dispatch(str_hecras_controller)

hec.ShowRas()
#hec.Compute_HideComputationWindow()

# Run the active HEC-RAS project 
hec.Project_Open(str_ras_projectpath)   # opening HEC-RAS

# ---------
# dictionary of this current HEC-RAS run
dict_hec_info = {
    "HEC-RAS_Version": hec.HECRASVersion(),
    "Project_Path": hec.CurrentProjectFile(),
    "Project_Title": hec.CurrentProjectTitle(),
    "Plan_Path": hec.CurrentPlanFile(),
    "Geometry_Path": hec.CurrentGeomFile(),
    "Geometry_HDF_Path": hec.CurrentGeomHDFFile(),
    "Unsteady_File_Path": hec.CurrentUnsteadyFile()
}

# name of 2d areas in active geometry
tup_2dareas = hec.Geometry_Get2DFlowAreas()

# Extracting the string part, stripping whitespace, and converting it to a list of strings
list_of_strings_2darea = [item.strip() for sublist in tup_2dareas if isinstance(sublist, tuple) for item in sublist]

# Add the list_of_strings_2darea to the dict_hec_info under a suitable key
dict_hec_info["2D_Flow_Area_Names"] = list_of_strings_2darea
# ---------

In [4]:
#dict_hec_info

In [5]:
# HEC-RAS should create a matching cXX for the geometry file gXX
str_expected_c_file = fn_filepath_expected(hec.CurrentGeomFile(), 'g', 'c')

# HEC-RAS should create a matching bXX for the geometry file pXX
str_expected_b_file = fn_filepath_expected(hec.CurrentPlanFile(), 'p', 'b')

# HDF Version of the plan
str_plan_hdf_filepath = hec.CurrentPlanFile() + ".tmp.hdf"

# create a list of the expected files
list_expected_files = [str_expected_c_file, str_expected_b_file, str_plan_hdf_filepath]

In [12]:
#list_files_exists

In [13]:
#list_expected_files

In [9]:
# Blocking mode set to false, so we can check file creation while running
is_blocking_mode = False

# option to kill HEC-RAS
b_kill_hecras = False

# to be populated: number and list of messages, blocking mode
NMsg, TabMsg, block = None, None, is_blocking_mode

# computations of the current plan
v1, NMsg, TabMsg, v2 = hec.Compute_CurrentPlan(NMsg, TabMsg, is_blocking_mode)


start_time = time.time()
while not all(os.path.exists(file_path) for file_path in list_expected_files) or b_kill_hecras:
    list_files_exists = [os.path.exists(file_path) for file_path in list_expected_files]

    if all(list_files_exists):
        # All the files exist
        print(f'Time taken: {time.time() - start_time:.2f} seconds')
        b_kill_hecras = True
        hec.Compute_Cancel()
        #hec.QuitRas()
        break  # Exit the loop once all files exist

print('Files Created')
print(f'Time taken: {time.time() - start_time:.2f} seconds')

#hec.QuitRas()  # close HEC-RAS

In [10]:
hec.QuitRas()